In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [ ]:
train = pd.read_csv('/content/train_dataset.csv')
test = pd.read_csv('/content/test_dataset.csv')

Tarkib:

Gender: yo'lovchilarning jinsi (ayol, erkak)

Customer Type: mijoz turi (sodiq mijoz, ishonchsiz mijoz)

Age: yo'lovchilarning haqiqiy yoshi

Type of Travel: yo'lovchilar parvozining maqsadi (shaxsiy sayohat, biznes sayohat)

Class: yo'lovchilar samolyotida sayohat klassi (Business, Eco, Eco Plus)

Flight distance: Ushbu sayohatning parvoz masofasi

Inflight wifi service: Parvoz ichidagi Wi-Fi xizmatidan qoniqish darajasi (0: Tegishli emas; 1-5)

Departure/Arrival time convenient: Ketish/Kelish vaqtining qoniqish darajasi

Ease of Online booking: Onlayn bron qilishdan qoniqish darajasi

Gate location: Darvoza joylashuvidan qoniqish darajasi

Food and drink: Oziq-ovqat va ichimlikdan qoniqish darajasi

Online boarding: Onlayn bortdan qoniqish darajasi

Seat comfort: O'rindiqning qulayligidan qoniqish darajasi

Inflight entertainment: Parvoz ichidagi o'yin-kulgidan qoniqish darajasi

On-board service: Bort xizmatidan qoniqish darajasi

Leg room service: oyoq xonasi xizmatidan qoniqish darajasi

Baggage handling: bagajni tashishdan qoniqish darajasi

Check-in service: Ro'yxatdan o'tish xizmatidan qoniqish darajasi

Inflight service: Parvoz ichidagi xizmatdan qoniqish darajasi

Cleanliness: Tozalikdan qoniqish darajasi

Departure Delay in Minutes: jo‘nash vaqtida kechikish daqiqalari

Arrival Delay in Minutes: yetib kelganda kechikish daqiqalari

Satisfaction: Aviakompaniyaning qoniqish darajasi (qoniqish, neytral yoki norozilik)

In [ ]:
train.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,Male,disloyal Customer,33,Business travel,Eco,571,2,3,2,...,4,3,1,3,4,3,4,10,3.0,0
1,2,Female,Loyal Customer,49,Business travel,Business,1431,4,1,4,...,5,5,5,5,3,5,3,0,0.0,1
2,3,Female,Loyal Customer,43,Business travel,Eco,867,1,4,4,...,1,1,1,1,1,1,2,0,18.0,0
3,4,Female,Loyal Customer,27,Business travel,Business,1550,3,3,3,...,2,4,4,5,5,4,2,0,0.0,1
4,5,Male,Loyal Customer,11,Personal Travel,Eco,526,3,4,3,...,4,5,2,5,3,5,4,0,10.0,0


In [ ]:
test.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,1,Female,Loyal Customer,25,Personal Travel,Eco,2704,2,2,2,...,5,2,4,4,3,5,1,5,152,191.0
1,2,Female,Loyal Customer,49,Business travel,Business,1623,1,1,1,...,4,2,2,2,2,4,2,3,60,52.0
2,3,Male,Loyal Customer,51,Business travel,Business,338,4,4,4,...,5,5,5,5,5,4,5,4,0,0.0
3,4,Male,Loyal Customer,34,Business travel,Eco Plus,95,0,0,0,...,1,5,4,4,1,3,3,5,0,0.0
4,5,Male,Loyal Customer,34,Personal Travel,Eco,602,4,4,4,...,5,5,4,5,4,5,5,5,130,140.0


In [ ]:
print(train.shape)
print(test.shape)

(10000, 24)
(4000, 23)


In [ ]:
train.dropna(inplace=True)
test.fillna(test["Arrival Delay in Minutes"].mean(),inplace=True)

In [ ]:
train.columns

Index(['id', 'Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [ ]:
train.corr()["satisfaction"].abs().sort_values()

<ipython-input-214-858fc2e5af50>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()["satisfaction"].abs().sort_values()


id                                   0.002581
Gate location                        0.007422
Departure Delay in Minutes           0.043456
Arrival Delay in Minutes             0.048650
Departure/Arrival time convenient    0.067758
Age                                  0.145327
Ease of Online booking               0.164025
Food and drink                       0.206881
Checkin service                      0.232337
Inflight service                     0.240580
Baggage handling                     0.248839
Inflight wifi service                0.277497
Flight Distance                      0.298484
Cleanliness                          0.310046
Leg room service                     0.325405
On-board service                     0.327361
Seat comfort                         0.342061
Inflight entertainment               0.401109
Online boarding                      0.504671
satisfaction                         1.000000
Name: satisfaction, dtype: float64

In [ ]:
train.drop(['id'], axis=1, inplace=True)
test.drop(["id"], axis=1, inplace=True)

In [ ]:
train

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,disloyal Customer,33,Business travel,Eco,571,2,3,2,4,...,4,3,1,3,4,3,4,10,3.0,0
1,Female,Loyal Customer,49,Business travel,Business,1431,4,1,4,4,...,5,5,5,5,3,5,3,0,0.0,1
2,Female,Loyal Customer,43,Business travel,Eco,867,1,4,4,4,...,1,1,1,1,1,1,2,0,18.0,0
3,Female,Loyal Customer,27,Business travel,Business,1550,3,3,3,3,...,2,4,4,5,5,4,2,0,0.0,1
4,Male,Loyal Customer,11,Personal Travel,Eco,526,3,4,3,2,...,4,5,2,5,3,5,4,0,10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Female,Loyal Customer,54,Personal Travel,Eco,1068,1,4,1,2,...,4,4,1,4,4,4,3,21,12.0,0
9996,Female,Loyal Customer,54,Business travel,Eco Plus,1399,4,1,1,1,...,4,4,4,4,1,4,4,34,21.0,0
9997,Male,Loyal Customer,46,Business travel,Business,1440,3,3,2,3,...,4,4,4,4,3,4,3,37,38.0,1
9998,Female,Loyal Customer,41,Business travel,Business,90,4,1,4,4,...,3,4,5,5,5,4,5,64,210.0,1


In [ ]:
x = train.drop('satisfaction',axis=1)
y = train['satisfaction']

In [ ]:
nums = ['Age','Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',]
cats = ['Gender', 'Customer Type',"Class", "Type of Travel"]

num_pip = Pipeline([
    ('MinMax', MinMaxScaler())
])

cat_pip = Pipeline([
    ("ohe", OneHotEncoder())
])

full_pip = ColumnTransformer([
    ('nums', num_pip, nums),
    ('cats', cat_pip, cats)
])

In [ ]:
x = full_pip.fit_transform(x)

In [ ]:
model_rf = RandomForestClassifier()

In [ ]:
model_rf.fit(x,y)

RandomForestClassifier()

In [ ]:
model_dt = DecisionTreeClassifier()

In [ ]:
model_dt.fit(x,y)

DecisionTreeClassifier()

In [ ]:
model_xgbc = XGBClassifier()
model_xgbc.fit(x,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
train_predicted_xgbc = model_xgbc.predict(x)

In [ ]:
train_predicted_rf = model_rf.predict(x)

In [ ]:
train_predicted_ds = model_rf.predict(x)

In [ ]:
train["train_predicted_rf"] = train_predicted_rf
train['train_predicted_ds'] = train_predicted_ds
train

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction,train_predicted_rf,train_predicted_ds
0,Male,disloyal Customer,33,Business travel,Eco,571,2,3,2,4,...,1,3,4,3,4,10,3.0,0,0,0
1,Female,Loyal Customer,49,Business travel,Business,1431,4,1,4,4,...,5,5,3,5,3,0,0.0,1,1,1
2,Female,Loyal Customer,43,Business travel,Eco,867,1,4,4,4,...,1,1,1,1,2,0,18.0,0,0,0
3,Female,Loyal Customer,27,Business travel,Business,1550,3,3,3,3,...,4,5,5,4,2,0,0.0,1,1,1
4,Male,Loyal Customer,11,Personal Travel,Eco,526,3,4,3,2,...,2,5,3,5,4,0,10.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Female,Loyal Customer,54,Personal Travel,Eco,1068,1,4,1,2,...,1,4,4,4,3,21,12.0,0,0,0
9996,Female,Loyal Customer,54,Business travel,Eco Plus,1399,4,1,1,1,...,4,4,1,4,4,34,21.0,0,0,0
9997,Male,Loyal Customer,46,Business travel,Business,1440,3,3,2,3,...,4,4,3,4,3,37,38.0,1,1,1
9998,Female,Loyal Customer,41,Business travel,Business,90,4,1,4,4,...,5,5,5,4,5,64,210.0,1,1,1


In [ ]:
print(train['satisfaction'].sum())
print(train['train_predicted_rf'].sum())
print(train['train_predicted_ds'].sum())

4986
4986
4986


In [ ]:
test = full_pip.transform(test)

In [ ]:
test

array([[0.24657534, 0.53978191, 0.4       , ..., 0.        , 0.        ,
        1.        ],
       [0.57534247, 0.32148627, 0.2       , ..., 0.        , 1.        ,
        0.        ],
       [0.60273973, 0.06199515, 0.8       , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.42465753, 0.11530695, 0.4       , ..., 0.        , 1.        ,
        0.        ],
       [0.21917808, 0.19123586, 1.        , ..., 0.        , 1.        ,
        0.        ],
       [0.24657534, 0.21304523, 0.6       , ..., 0.        , 1.        ,
        0.        ]])

In [ ]:
comp_pre_data_ds = model_dt.predict(test)
comp_pre_data_rf = model_rf.predict(test)
comp_pre_data_x = model_xgbc.predict(test)

In [ ]:
comp_1 = pd.read_csv("/content/sample_submission.csv")
comp_2 = comp_1.copy()
comp_3 = comp_1.copy()

In [ ]:
comp_1['satisfaction'] = comp_pre_data_ds
comp_2['satisfaction'] = comp_pre_data_rf

In [ ]:
comp_3['satisfaction'] = comp_pre_data_x

In [ ]:
comp_1.to_csv('completed_1.csv',index=False)
comp_2.to_csv("completed_2.csv", index=False)
comp_3.to_csv("completed_3.csv", index=False)

In [ ]:
comp_3

,id,satisfaction
0,1,0
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
3995,3996,1
3996,3997,0
3997,3998,0
3998,3999,1
